In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 100

import seaborn as sns
import pandas_profiling as pdp
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import collections
import re

from sklearn.preprocessing import LabelEncoder

In [5]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
sample=pd.read_csv('../input/sample_submit.csv')

train_index=len(train)
test_index=len(test)
df_all=pd.concat([train,test],axis=0,sort=False).reset_index(drop=True)

df_all.columns=['id','rent','address','access','floor_info','old','direction','square','floor','bath','kitchen','broadcast','facility','parking','neighbors','structure','period']
df_all[:1]

,id,rent,address,access,floor_info,old,direction,square,floor,bath,kitchen,broadcast,facility,parking,neighbors,structure,period
0,1,75000.0,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間


## 所在地を区、町で分ける

In [6]:
# 住所から区を取得
df_all['address']=df_all['address'].apply(lambda x:x.split('都')[1])
df_all['address_city']=df_all['address'].apply(lambda x:x.split('区')[0])

le_city=LabelEncoder()
df_all['address_city']=le_city.fit_transform(df_all['address_city'])
df_all['address_city']=df_all['address_city'].astype('category')

# 住所から町名を取得
town_tmp=df_all['address'].apply(lambda x:x.split('区')[1])
df_all['address_town']=town_tmp.apply(lambda x:re.split(r'\d+',x)[0])

miss_town=[w  for w in df_all['address_town'].unique() if '丁目'in w]
print(miss_town)

def clean_town(src):
    if src in miss_town:
        return re.split('[一,二,四]丁目',src)[0]
    else:
        return src
    
df_all['address_town']=df_all['address_town'].apply(lambda x:clean_town(x))

le_town=LabelEncoder()
df_all['address_town']=le_town.fit_transform(df_all['address_town'])
df_all['address_town']=df_all['address_town'].astype('category')

['太子堂一丁目', '本羽田一丁目', '東品川四丁目', '大森北一丁目', '八雲二丁目']


## アクセス特徴の一つから路線と駅名、距離をとる


In [7]:
#  アクセス情報の初めの要素から路線と駅名をとる
df_all['train_line']=df_all['access'].apply(lambda x:x.split('\t')[0])
df_all['train_station']=df_all['access'].apply(lambda x:x.split('\t')[1])

# バス移動のものはバス特徴量にまとめる
df_all['station_access']=df_all['access'].apply(lambda x:x.split('\t')[2])
df_all['station_access']=df_all['station_access'].apply(lambda x:'車移動' if 'バス' in x or '車' in x else x)

#  数値型に変更
def enc_num(x):
    if x=='車移動':
        return 999
    else:
        return int(re.findall(f'\d+',x)[0])
    
df_all['station_access']=df_all['station_access'].apply(lambda x:enc_num(x))

for col in ['train_line','train_station']:
    le=LabelEncoder()
    df_all[col]=le.fit_transform(df_all[col])
    df_all[col]=df_all[col].astype('category')

##  フロア情報をカテゴリエンコ

In [8]:
le_floor_info=LabelEncoder()
le_floor_info.fit(df_all['floor_info'])
df_all['floor_cat']=le_floor_info.transform(df_all['floor_info'])
df_all['floor_cat']=df_all['floor_cat'].astype('category')

##  築年数を月単位の経過日数に変換

In [9]:
df_all['old_num']=df_all['old'].apply(lambda x:int(x.split('年')[0])*12+int(x.split('年')[1].split('ヶ月')[0])+1 if x!='新築' else 0)

##  方角をカテゴリエンコ、欠損を最頻値の南で補完


In [10]:
pd.DataFrame(df_all['direction'].value_counts())
print('欠損数:',df_all['direction'].isnull().sum())
df_all['direction'].fillna('南',inplace=True)

le_direction=LabelEncoder()
df_all['direction_cat']=le_direction.fit_transform(df_all['direction'])
df_all['direction_cat']=df_all['direction_cat'].astype('category')

欠損数: 5557


## 面積を数値化

In [11]:
df_all['square_num']=df_all['square'].apply(lambda x:float(x.split('m')[0]))

## 部屋の階、何階建てか、地下室の有無を数値化


In [12]:
def room_floor(src):
    tmp=src.split('／')
    # floor_infoは２分割までできるので、分割できる場合は、一つ目の要素が部屋の階
    if len(tmp)==2:
        return tmp[0]
    
    if len(tmp)==1:
        if '階建' in tmp[0]:
             return -999
        else:
            return tmp[0]

def room_floor_enc(x):
    #  数字部分の抜き出し
    if x==-999:
        return -999
    elif x=='':
        return -999
    
    else:

        num=re.findall(r'\d+',x)[0]
    #  地下の場合はマイナスにする
    if '地下' in x:
        return -1*int(num)
    else:
        return int(num)
    
 #  部屋の階を数値化
df_all['floor']=df_all['floor'].fillna('2階／2階建')        
df_all['room_floor']=df_all['floor'].apply(lambda x:room_floor(x))
df_all['room_floor']=df_all['room_floor'].apply(lambda x:room_floor_enc(x))


In [13]:
def building_floor(src):
    tmp=src.split('／')
    # floor_infoは２分割までできるので、分割できる場合は、一つ目の要素が部屋の階
    if len(tmp)==2:
        return tmp[1]
    
    if len(tmp)==1:
        if '階建' in tmp[0]:
             return tmp[0]
        else:
            return -999

# 何階建かを取得
def building_floor_enc(x):
    if x==-999:
        return -999
        
    else:
        return int(x.split('階建')[0])
    
# 　地下の階数を取得
def check_underground(x):
# 欠損部分は、0を埋める
    if x==-999:
        return 0
    else:
        under_info=re.findall('地下+\d',x)
        if len(under_info)==0:
            return 0
        else:
            return int(re.findall('地下+\d',x)[0][-1])
    
df_all['floor']=df_all['floor'].fillna('2階／2階建')        
df_all['building_floor']=df_all['floor'].apply(lambda x:building_floor(x))

# 地下特徴量を加える
df_all['underground']=0
df_all['underground']=df_all['building_floor'].apply(lambda x:check_underground(x))

# 建物の階数を加える
df_all['building_floor']=df_all['building_floor'].apply(lambda x:building_floor_enc(x))

## お風呂・トイレの設備をワンホットエンコ

In [14]:
df_all['bath']=df_all['bath'].fillna(0)
df_all['bath_list']=df_all['bath'].apply(lambda x:x.split('／') if x !=0 else 'nan')

def drop_t(x):
    if x!='nan':
        return list(map(lambda y:y.strip('\t'),x))
    else:
        return 0
    
df_all['bath_list']=df_all['bath_list'].apply(lambda x:drop_t(x))

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

words=[]

# CountVectorizer用の文字列リストを作成
for i in range(len(df_all)):
    str_list=df_all['bath_list'][i]
    
    # 要素が０の時は空白文字
    if str_list==0:
        words.append('')
        
    # リストを文字列化
    else:
        maped_list = map(str, str_list)  #mapで要素すべてを文字列に
        mojiretu = ','.join(maped_list)
        words.append(mojiretu)

# CountVectorizer
vec_count = CountVectorizer()

# ベクトル化
vec_count.fit(words)
X = vec_count.transform(words)

print('Vocabulary size: {}'.format(len(vec_count.vocabulary_)))
print('Vocabulary content: {}'.format(vec_count.vocabulary_))
bath_df=pd.DataFrame(X.toarray(), columns=vec_count.get_feature_names())
bath_df.drop(['バス'],axis=1,inplace=True)
df_all=pd.concat([df_all,bath_df],axis=1)

Vocabulary size: 14
Vocabulary content: {'専用バス': 8, '専用トイレ': 7, 'バス': 3, 'トイレ別': 2, 'シャワー': 0, '浴室乾燥機': 10, '温水洗浄便座': 11, '洗面台独立': 9, '脱衣所': 12, '追焚機能': 13, '共同トイレ': 5, 'バスなし': 4, '共同バス': 6, 'トイレなし': 1}


## キッチン情報をワンホットエンコ

In [16]:
df_all['kitchen']=df_all['kitchen'].fillna(0)
df_all['kitchen_list']=df_all['kitchen'].apply(lambda x:x.split('／') if x !=0 else 'nan')

def drop_t(x):
    if x!='nan':
        return list(map(lambda y:y.strip('\t'),x))
    else:
        return 0
    
df_all['kitchen_list']=df_all['kitchen_list'].apply(lambda x:drop_t(x))

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

words=[]

# CountVectorizer用の文字列リストを作成
for i in range(len(df_all)):
    str_list=df_all['kitchen_list'][i]
    
    # 要素が０の時は空白文字
    if str_list==0:
        words.append('')
        
    # リストを文字列化
    else:
        maped_list = map(str, str_list)  #mapで要素すべてを文字列に
        mojiretu = ','.join(maped_list)
        words.append(mojiretu)

# CountVectorizer
vec_count = CountVectorizer()

# ベクトル化
vec_count.fit(words)
X = vec_count.transform(words)

print('Vocabulary size: {}'.format(len(vec_count.vocabulary_)))
print('Vocabulary content: {}'.format(vec_count.vocabulary_))
kitchen_df=pd.DataFrame(X.toarray(), columns=vec_count.get_feature_names())

df_all=pd.concat([df_all,kitchen_df],axis=1)

Vocabulary size: 15
Vocabulary content: {'ガスコンロ': 3, 'コンロ2口': 5, 'システムキッチン': 9, '給湯': 13, '独立キッチン': 12, 'コンロ3口': 6, 'ihコンロ': 0, 'コンロ1口': 4, '冷蔵庫あり': 10, 'コンロ設置可': 8, 'カウンターキッチン': 2, 'l字キッチン': 1, '口数不明': 11, '電気コンロ': 14, 'コンロ4口以上': 7}


## 通信機器情報をワンホットエンコ

In [18]:
df_all['broadcast']=df_all['broadcast'].fillna(0)
df_all['broadcast_list']=df_all['broadcast'].apply(lambda x:x.split('／') if x !=0 else 'nan')

def drop_t(x):
    if x!='nan':
        return list(map(lambda y:y.strip('\t'),x))
    else:
        return 0
    
df_all['broadcast_list']=df_all['broadcast_list'].apply(lambda x:drop_t(x))

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

words=[]

# CountVectorizer用の文字列リストを作成
for i in range(len(df_all)):
    str_list=df_all['broadcast_list'][i]
    
    # 要素が０の時は空白文字
    if str_list==0:
        words.append('')
        
    # リストを文字列化
    else:
        maped_list = map(str, str_list)  #mapで要素すべてを文字列に
        mojiretu = ','.join(maped_list)
        words.append(mojiretu)

# CountVectorizer
vec_count = CountVectorizer()

# ベクトル化
vec_count.fit(words)
X = vec_count.transform(words)

print('Vocabulary size: {}'.format(len(vec_count.vocabulary_)))
print('Vocabulary content: {}'.format(vec_count.vocabulary_))
broadcast_df=pd.DataFrame(X.toarray(), columns=vec_count.get_feature_names())

df_all=pd.concat([df_all,broadcast_df],axis=1)

Vocabulary size: 8
Vocabulary content: {'インターネット対応': 4, 'catv': 1, 'csアンテナ': 2, 'bsアンテナ': 0, '光ファイバー': 5, '高速インターネット': 7, 'インターネット使用料無料': 3, '有線放送': 6}


## 設備をワンホットエンコ

In [20]:
df_all['facility']=df_all['facility'].fillna(0)
df_all['facility_list']=df_all['facility'].apply(lambda x:x.split('\t') if x !=0 else 'nan')

def clean_list(words):
    if words=='nan':
        return 'nan'
    else:
        return [w for w in words if w !='／']

df_all['facility_list']=df_all['facility_list'].apply(lambda x:clean_list(x))

def drop_t(x):
    if x!='nan':
        return list(map(lambda y:y.strip('／'),x))
    else:
        return 0
    
df_all['facility_list']=df_all['facility_list'].apply(lambda x:drop_t(x))

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

words=[]

# CountVectorizer用の文字列リストを作成
for i in range(len(df_all)):
    str_list=df_all['facility_list'][i]
    
    # 要素が０の時は空白文字
    if str_list==0:
        words.append('')
        
    # リストを文字列化
    else:
        maped_list = map(str, str_list)  #mapで要素すべてを文字列に
        mojiretu = ','.join(maped_list)
        words.append(mojiretu)

# CountVectorizer
vec_count = CountVectorizer()

# ベクトル化
vec_count.fit(words)
X = vec_count.transform(words)

print('Vocabulary size: {}'.format(len(vec_count.vocabulary_)))
print('Vocabulary content: {}'.format(vec_count.vocabulary_))
facility_df=pd.DataFrame(X.toarray(), columns=vec_count.get_feature_names())

df_all=pd.concat([df_all,facility_df],axis=1)

Vocabulary size: 42
Vocabulary content: {'エアコン付': 4, 'シューズボックス': 10, 'バルコニー': 14, 'フローリング': 15, '室内洗濯機置場': 28, '敷地内ごみ置き場': 34, 'エレベーター': 5, '公営水道': 24, '下水': 20, '都市ガス': 40, 'タイル張り': 11, 'ウォークインクローゼット': 3, '2面採光': 1, '24時間換気システム': 0, '3面採光': 2, 'ペアガラス': 17, '専用庭': 30, '水道その他': 35, '冷房': 25, 'クッションフロア': 9, '床暖房': 32, 'プロパンガス': 16, 'ロフト付き': 19, '出窓': 26, 'トランクルーム': 12, '汲み取り': 36, 'オール電化': 6, 'ルーフバルコニー': 18, '室外洗濯機置場': 29, '床下収納': 31, 'バリアフリー': 13, '浄化槽': 38, '防音室': 41, '二重サッシ': 22, '二世帯住宅': 21, 'ガスその他': 7, '洗濯機置場なし': 37, '排水その他': 33, '石油暖房': 39, '地下室': 27, 'ガス暖房': 8, '井戸': 23}


## 駐車場の料金が分かるものは値を入れる

In [22]:
df_all['parking']=df_all['parking'].fillna(0)
price=[]
def parking_price(src):
    #  欠損は０とする
    if src==0:
        return 0
    # 何かしらの料金の値を入れる。
    else:
        tmp=re.findall(r'\d+,\d+円',src)
        if len(tmp)==0:
            return 0
    # 料金が複数ある場合は最大値を入れる
        else:
            if len(tmp)==1:
                nums=tmp[0][:-1].split(',')
                return int(nums[0])*1000+int(nums[1])
            else:
                if len(tmp)==2:
                    nums1=tmp[0][:-1].split(',')
                    nums2=tmp[1][:-1].split(',')
            
                    return max(int(nums1[0])*1000+int(nums1[1]),int(nums2[0])*1000+int(nums2[1]))
            
                else:

                    nums1=tmp[0][:-1].split(',')
                    nums2=tmp[1][:-1].split(',')
                    nums3=tmp[2][:-1].split(',')
                    return max(int(nums1[0])*1000+int(nums1[1]),int(nums2[0])*1000+int(nums2[1]),int(nums3[0])*1000+int(nums3[1]))

df_all['parking_price']=df_all['parking'].apply(lambda x:parking_price(x))

## 近隣状況の距離を入れる

In [23]:
df_all['neighbors']=df_all['neighbors'].fillna(0)
neightbor_dict=[]
convenience_count=[]
supermarket_count=[]

for i in tqdm(range(len(df_all))):
    tmp=dict()
    convenience=0
    supermarket=0
    
    neighbor_info=df_all['neighbors'][i]
    
    # 近隣情報が0 の場合は０を返す
    if neighbor_info==0:
        neightbor_dict.append(0)
        convenience_count.append(convenience)
        supermarket_count.append(supermarket)
        
    # 近隣情報の辞書配列を作成
    else:   
        for word in neighbor_info.split('\t'):
            place = re.findall(r'\【.+?\】', word)[0][1:-1]  # 建物の名前
            distance=int(re.findall(r'\d+m', word)[0][:-1])  #  距離
    
    # すでに同じ建物がある場合は、近い距離の値を入れる
            if place in tmp.keys():
                tmp[place]=max(tmp[place],distance)
            else:
                tmp[place]=distance
                
    #  コンビニとスーパーの数をカウント
            if place=='コンビニ':
                convenience+=1
            if place=='スーパー':
                supermarket+=1
        
        neightbor_dict.append(tmp)
        convenience_count.append(convenience)
        supermarket_count.append(supermarket)

100%|██████████| 62732/62732 [00:01<00:00, 39454.68it/s]


In [26]:
df_all['neighbor_dict']=neightbor_dict
df_all['convenience_count']=convenience_count
df_all['supermarket_count']=supermarket_count

In [27]:
# 辞書から指定された場所の距離を取得
def neighbor_distance(x,place):
    if x==0:
        return 0
    elif place in x.keys():
        return x[place]
    else:
        return 0

# 全ての場所の配列を取得
places=[]
for i in range(len(df_all)):
    neighbor=df_all['neighbor_dict'][i]
    if neighbor==0:
        continue
    else:
        places.extend(neighbor.keys())

for place in set(places):
    df_all[place]=df_all['neighbor_dict'].apply(lambda x:neighbor_distance(x,place))

## 構造をラベルエンコ

In [28]:
le_structure=LabelEncoder()
df_all['structure_cat']=le_structure.fit_transform(df_all['structure'])
df_all['structure_cat']=df_all['structure_cat'].astype('category')

## 期間を月単位に数値化

In [29]:
# 欠損を最頻値で補完
df_all['period']=df_all['period'].fillna('2年間')

# 定期借家かどうか
df_all['is_rent']=df_all['period'].apply(lambda x:1 if '定期借家' in x else 0)

In [30]:
# 期間を月単位に変換
def rent_period(x):
    if 'まで' in x:
        year=re.findall(r'\d+年',x)[0][:-1]
        month=re.findall(r'\d+月',x)[0][:-1]
        return int(year)*12+int(month)-2019*12-8
    
    else:
        year=re.findall(r'\d+年',x)
        year=0 if len(year)==0 else year[0][:-1]

        month=re.findall(r'\d+ヶ', x)
        month=0 if len(month)==0 else month[0][:-1]
        
        return int(year)*12+int(month)
        
df_all['period_num']=df_all['period'].apply(lambda x:rent_period(x.split('\t')[0]))

In [33]:
df_all=df_all.select_dtypes(exclude='object')
train=df_all[:train_index]
test=df_all[train_index:]

In [35]:
train.to_csv('train_feat1.csv')
test.to_csv('test_feat1.csv')

'2021'